In [18]:
import pandas as pd

# Load EUTL dataset
- Source
    - The reason why a same transaction (with one TRANSACTION_ID) is described by multiple lines is because the file details transactions by ORIGINATING_REGISTRY, UNIT_TYPE_DESCRIPTION, SUPP_UNIT_TYPE_DESCRIPTION, ORIGINAL_PERIOD_CODE, LULUCF_CODE_DESCRIPTION, PROJECT_IDENTIFIER, TRACK and EXPIRY_DATE, which can be many for one transaction
- Information of EUTL: X EUAs are trasnferred from account A to account B on date T
- [Abrell](https://www.euets.info/) support tools:  
    - [scraper](https://github.com/jabrell/eutl_scraper): provides access to the Python routines for downloading and processing of the source data
    - [pyeutl](https://github.com/jabrell/pyeutl/tree/main): provides Python routines to access the data provided in a convenient way
    - [transactions_analysis](https://github.com/jabrell/transaction_eutl)

## Transactions

### Check optimal dtypes
- based on number of unique values, set as category or string

In [2]:
df_eutl_2022 = pd.read_csv(r'../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv', nrows = 10000)

for col in df_eutl_2022.select_dtypes(include="object"):
    print(f"{col}: {df_eutl_2022[col].nunique(dropna=False) / len(df_eutl_2022)}")

TRANSACTION_ID: 0.7958
TRANSACTION_TYPE: 0.0004
TRANSACTION_DATE: 0.79
TRANSACTION_STATUS: 0.0001
TRANSFERRING_REGISTRY_NAME: 0.0033
TRANSFERRING_ACCOUNT_TYPE2: 0.0004
TRANSFERRING_ACCOUNT_TYPE3: 0.0007
TRANSFERRING_ACCOUNT_OPEN_DT: 0.144
TRANSFERRING_ACCOUNT_END_OF_VALIDITY: 0.0655
TRANSFERRING_ACCOUNT_NAME: 0.2351
TRANSFERRING_ACCOUNT_HOLDER: 0.1922
TRANSFERRING_ACCOUNT_HOLDER_ADDRESS1: 0.1836
TRANSFERRING_ACCOUNT_HOLDER_ADDRESS2: 0.0475
TRANSFERRING_ACCOUNT_HOLDER_CITY: 0.1294
TRANSFERRING_ACCOUNT_HOLDER_POSTAL_CODE: 0.1552
TRANSFERRING_ACCOUNT_HOLDER_COUNTRY_CODE: 0.0045
TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER: 0.1879
TRANSFERRING_ACCOUNT_HOLDER_LEI: 0.0574
TRANSFERRING_INSTALLATION_NAME: 0.1836
TRANSFERRING_INSTALLATION_PERMIT_IDENTIFIER: 0.1862
TRANSFERRING_INSTALLATION_PARENT_COMPANY: 0.0399
TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY: 0.0111
TRANSFERRING_INSTALLATION_EPER_IDENTIFICATION: 0.0633
TRANSFERRING_INSTALLATION_CITY: 0.1476
TRANSFERRING_INSTALLATION_PO

C:\Users\ATapia\AppData\Local\Temp\ipykernel_20784\203569506.py:1: DtypeWarning: Columns (0: LULUCF_CODE_DESCRIPTION) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eutl_2022 = pd.read_csv(r'../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv', nrows = 10000)
C:\Users\ATapia\AppData\Local\Temp\ipykernel_20784\203569506.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  for col in df_eutl_2022.select_dtypes(include="object"):


In [3]:
dtypes = {
    # IDs / keys
    "TRANSACTION_ID": "string",

    # Low-cardinality / codes
    "TRANSACTION_TYPE": "category",
    "TRANSACTION_STATUS": "category",

    "TRANSFERRING_REGISTRY_NAME": "category",
    "ACQUIRING_REGISTRY_NAME": "category",
    "ORIGINATING_REGISTRY": "category",

    "TRANSFERRING_ACCOUNT_TYPE2": "category",
    "TRANSFERRING_ACCOUNT_TYPE3": "category",
    "ACQUIRING_ACCOUNT_TYPE2": "category",
    "ACQUIRING_ACCOUNT_TYPE3": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_COUNTRY_CODE": "category",
    "ACQUIRING_ACCOUNT_HOLDER_COUNTRY_CODE": "category",

    "TRANSFERRING_INSTALLATION_MAIN_ACTIVITY": "category",
    "ACQUIRING_INSTALLATION_MAIN_ACTIVITY": "category",

    "UNIT_TYPE_DESCRIPTION": "category",
    "SUPP_UNIT_TYPE_DESCRIPTION": "category",
    "LULUCF_CODE_DESCRIPTION": "string",  # force away mixed types; you can later .astype("category") if you want

    # Lower-cardinality text (often worth category)
    "TRANSFERRING_INSTALLATION_PARENT_COMPANY": "category",
    "TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY": "category",
    "ACQUIRING_INSTALLATION_PARENT_COMPANY": "category",
    "ACQUIRING_INSTALLATION_SUBSIDIARY_COMPANY": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_LEI": "category",
    "ACQUIRING_ACCOUNT_HOLDER_LEI": "category",

    "TRANSFERRING_ACCOUNT_HOLDER_ADDRESS2": "category",
    "ACQUIRING_ACCOUNT_HOLDER_ADDRESS2": "category",
    "TRANSFERRING_INSTALLATION_ADDRESS2": "category",
    "ACQUIRING_INSTALLATION_ADDRESS2": "category",

    "TRANSFERRING_INSTALLATION_EPER_IDENTIFICATION": "category",
    "ACQUIRING_INSTALLATION_EPER_IDENTIFICATION": "category",

    # High-ish cardinality text -> string (safe default)
    "TRANSFERRING_ACCOUNT_NAME": "string",
    "TRANSFERRING_ACCOUNT_HOLDER": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_ADDRESS1": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_CITY": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_POSTAL_CODE": "string",
    "TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER": "string",

    "TRANSFERRING_INSTALLATION_NAME": "string",
    "TRANSFERRING_INSTALLATION_PERMIT_IDENTIFIER": "string",
    "TRANSFERRING_INSTALLATION_CITY": "string",
    "TRANSFERRING_INSTALLATION_POSTAL_CODE": "string",
    "TRANSFERRING_INSTALLATION_ADDRESS1": "string",

    "ACQUIRING_ACCOUNT_NAME": "string",
    "ACQUIRING_ACCOUNT_HOLDER": "string",
    "ACQUIRING_ACCOUNT_HOLDER_ADDRESS1": "string",
    "ACQUIRING_ACCOUNT_HOLDER_CITY": "string",
    "ACQUIRING_ACCOUNT_HOLDER_POSTAL_CODE": "string",
    "ACQUIRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER": "string",

    "ACQUIRING_INSTALLATION_NAME": "string",
    "ACQUIRING_INSTALLATION_PERMIT_IDENTIFIER": "string",
    "ACQUIRING_INSTALLATION_CITY": "string",
    "ACQUIRING_INSTALLATION_POSTAL_CODE": "string",
    "ACQUIRING_INSTALLATION_ADDRESS1": "string",
}

### Set columns of interest and read

In [4]:
cols = ['TRANSACTION_ID', 'TRANSACTION_TYPE', 'TRANSACTION_DATE', 'TRANSFERRING_REGISTRY_NAME','TRANSFERRING_ACCOUNT_NAME',
       'TRANSFERRING_ACCOUNT_IDENTIFIER', 'TRANSFERRING_ACCOUNT_HOLDER', 'TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER',
       'TRANSFERRING_ACCOUNT_HOLDER_LEI', 'TRANSFERRING_INSTALLATION_NAME',
       'TRANSFERRING_INSTALLATION_INSTALLATION_IDENTIFIER','TRANSFERRING_INSTALLATION_PARENT_COMPANY',
       'TRANSFERRING_INSTALLATION_SUBSIDIARY_COMPANY','TRANSFERRING_INSTALLATION_MAIN_ACTIVITY', 'ACQUIRING_REGISTRY_NAME','ACQUIRING_ACCOUNT_NAME',
       'ACQUIRING_ACCOUNT_IDENTIFIER', 'ACQUIRING_ACCOUNT_HOLDER','ACQUIRING_ACCOUNT_HOLDER_LEI', 'ACQUIRING_INSTALLATION_NAME',
       'ACQUIRING_INSTALLATION_INSTALLATION_IDENTIFIER','ACQUIRING_INSTALLATION_PARENT_COMPANY',
       'ACQUIRING_INSTALLATION_SUBSIDIARY_COMPANY']

In [5]:
df_eutl_2022 = pd.read_csv(
    "../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv",
    usecols=cols,
    dtype=dtypes,
    parse_dates=["TRANSACTION_DATE"],
    low_memory=False    # internal chunksize is not customizable in python, so in order to avoid mixed types we need to disable it entirely
)

In [6]:
df_eutl_2022.info(memory_usage="deep")

<class 'pandas.DataFrame'>
RangeIndex: 2142475 entries, 0 to 2142474
Data columns (total 23 columns):
 #   Column                                                   Dtype         
---  ------                                                   -----         
 0   TRANSACTION_ID                                           string        
 1   TRANSACTION_TYPE                                         category      
 2   TRANSACTION_DATE                                         datetime64[us]
 3   TRANSFERRING_REGISTRY_NAME                               category      
 4   TRANSFERRING_ACCOUNT_NAME                                string        
 5   TRANSFERRING_ACCOUNT_IDENTIFIER                          float64       
 6   TRANSFERRING_ACCOUNT_HOLDER                              string        
 7   TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER  string        
 8   TRANSFERRING_ACCOUNT_HOLDER_LEI                          category      
 9   TRANSFERRING_INSTALLATION_NAME                

In [28]:
df_eutl_2022

,TRANSACTION_ID,TRANSACTION_TYPE,TRANSACTION_DATE,TRANSFERRING_REGISTRY_NAME,TRANSFERRING_ACCOUNT_NAME,TRANSFERRING_ACCOUNT_IDENTIFIER,TRANSFERRING_ACCOUNT_HOLDER,TRANSFERRING_ACCOUNT_HOLDER_COMPANY_REGISTRATION_NUMBER,TRANSFERRING_ACCOUNT_HOLDER_LEI,TRANSFERRING_INSTALLATION_NAME,...,TRANSFERRING_INSTALLATION_MAIN_ACTIVITY,ACQUIRING_REGISTRY_NAME,ACQUIRING_ACCOUNT_NAME,ACQUIRING_ACCOUNT_IDENTIFIER,ACQUIRING_ACCOUNT_HOLDER,ACQUIRING_ACCOUNT_HOLDER_LEI,ACQUIRING_INSTALLATION_NAME,ACQUIRING_INSTALLATION_INSTALLATION_IDENTIFIER,ACQUIRING_INSTALLATION_PARENT_COMPANY,ACQUIRING_INSTALLATION_SUBSIDIARY_COMPANY
0,DE122558,10-0,2015-06-04 17:38:44,Germany,1914 - RWE Power AG Personenkonto,1914.0,RWE Power Aktiengesellschaft,HRB 17420 Amtsgericht Essen,NaN,<NA>,...,-,Germany,FutureCamp Climate GmbH Personenkonto,2963,FutureCamp Climate GmbH,NaN,<NA>,NaN,NaN,NaN
1,GB66629,10-0,2012-08-28 15:29:28,United Kingdom,MLI Emissions Registry Account,901.0,Merrill Lynch International,02312079,NaN,<NA>,...,-,United Kingdom,MLCE,841,Merrill Lynch Commodities (Europe) Limited,NaN,<NA>,NaN,NaN,NaN
2,NL28978,10-0,2012-09-12 17:25:39,Netherlands,SIA Vidzeme Eko,286.0,"SIA ""Vidzeme Eko""",40003755312,NaN,<NA>,...,-,Netherlands,ACT Carbon,778,ACT Financial Solutions B.V.,724500LY73GPE4GDX159\r,<NA>,NaN,NaN,NaN
3,NL28981,10-0,2012-09-13 15:45:37,Netherlands,SIA Vidzeme Eko,286.0,"SIA ""Vidzeme Eko""",40003755312,NaN,<NA>,...,-,Netherlands,ACT Carbon,778,ACT Financial Solutions B.V.,724500LY73GPE4GDX159\r,<NA>,NaN,NaN,NaN
4,NL29004,10-0,2012-09-18 14:23:50,Netherlands,SIA Vidzeme Eko,286.0,"SIA ""Vidzeme Eko""",40003755312,NaN,<NA>,...,-,Netherlands,ACT Carbon,778,ACT Financial Solutions B.V.,724500LY73GPE4GDX159\r,<NA>,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2142470,EU542433,10-2,2020-01-09 18:14:20,Spain,"Tereos Starch & Sweeteners Iberia, S.A.U.",5008918.0,"Tereos Starch & Sweeteners Iberia, S.A.U.",A50012921,NaN,"Tereos Starch & Sweeteners Iberia, S.A.U.",...,20-Combustion of fuels,European Commission,EU Allowance deletion,5016380,European Commission,NaN,<NA>,NaN,NaN,NaN
2142471,FR93298,10-0,2009-05-07 17:26:34,France,CONSUS FR,1287.0,CONSUS FRANCE SARL,491808804,NaN,<NA>,...,-,France,CONSUS FR PWX,1291,KLAPUCKI,NaN,<NA>,NaN,NaN,NaN
2142472,FR10339,10-0,2006-06-01 16:34:39,France,Caisse des dépôts et consignations Détention,1163.0,Caisse des Dépôts et Consignations,180020026,NaN,<NA>,...,-,France,ACCORD ENERGY LTD Détention,1185,WINKLEHNER,NaN,<NA>,NaN,NaN,NaN
2142473,EU403486,10-2,2017-04-27 13:28:12,Romania,Operator Account,5010063.0,COMPLEXUL ENERGETIC HUNEDOARA,J20/994/2012,NaN,Electrocentrale Deva,...,20-Combustion of fuels,European Commission,EU Allowance deletion,5016380,European Commission,NaN,<NA>,NaN,NaN,NaN


In [37]:
df_eutl_2022.duplicated().sum()

np.int64(834084)

In [40]:
pd.read_csv(r'../data/raw/transactions_eutl_2025/transactions_EUTL_PUBLIC_NOTESD_20251031.csv', usecols=['UNIT_TYPE_DESCRIPTION']).value_counts()

UNIT_TYPE_DESCRIPTION                                        
AAU - Assigned Amount Unit                                       1054991
Non-Kyoto Unit                                                    708724
CER - Certified Emission Reduction Unit converted from an AAU     324244
ERU - Emission Reduction Unit                                      54135
tCER - Temporary CER                                                 217
RMU - Removal Unit                                                   130
ERU - Converted from an RMU                                           34
Name: count, dtype: int64

In [7]:
df_eutl_2022['TRANSACTION_DATE'].sort_values()

248759    2005-01-11 13:43:06
1394727   2005-02-01 11:17:29
1320551   2005-02-01 11:17:34
248858    2005-02-01 11:17:38
995909    2005-02-01 11:17:42
                  ...        
905217    2022-09-30 16:46:40
825280    2022-09-30 17:16:04
1317962   2022-09-30 17:25:07
1498982   2022-09-30 17:25:07
1523643   2022-09-30 17:30:38
Name: TRANSACTION_DATE, Length: 2142475, dtype: datetime64[us]

In [50]:
k = pd.read_csv(r"../data/raw/abrel_postgres_files/installations.csv")['installationID']

In [ ]:
k

## Compliance

In [23]:
pd.read_excel(r"../data/raw/compliance_2024_code_en.xlsx" ,nrows=0, header=1).columns.to_list()

['REGISTRY_CODE',
 'INSTALLATION_NAME',
 'INSTALLATION_IDENTIFIER',
 'PERMIT_IDENTIFIER',
 'MAIN_ACTIVITY_TYPE_CODE',
 'COMPLIANCE_CODE',
 'CH_COMPLIANCE_CODE',
 'COMPLIANCE_STATUS_LATEST_YEAR',
 'TOTAL_VERIFIED_EMISSIONS',
 'CH_TOTAL_VERIFIED_EMISSIONS',
 'TOTAL_SURRENDERED_ALLOWANCES',
 'YEAR_OF_FIRST_EMISSIONS',
 'YEAR_OF_LAST_EMISSIONS',
 'ACCOUNT_CLOSURE']

In [32]:
df_compliance = pd.read_excel(r"../data/raw/compliance_2024_code_en.xlsx" , header=1)

In [35]:
df_compliance.shape

(14573, 14)

In [44]:
df_compliance[df_compliance.duplicated(subset=['INSTALLATION_IDENTIFIER', 'REGISTRY_CODE'],keep=False)]

,REGISTRY_CODE,INSTALLATION_NAME,INSTALLATION_IDENTIFIER,PERMIT_IDENTIFIER,MAIN_ACTIVITY_TYPE_CODE,COMPLIANCE_CODE,CH_COMPLIANCE_CODE,COMPLIANCE_STATUS_LATEST_YEAR,TOTAL_VERIFIED_EMISSIONS,CH_TOTAL_VERIFIED_EMISSIONS,TOTAL_SURRENDERED_ALLOWANCES,YEAR_OF_FIRST_EMISSIONS,YEAR_OF_LAST_EMISSIONS,ACCOUNT_CLOSURE
6849,FR,3329,204199,3329-10/12/2009-V1,10,EXCLUDED SINCE 2021,EXCLUDED SINCE 2021,NaN,0,0,109,2012,NOT SET,OPEN
6850,FR,3329,204199,3329-10/12/2009-V1,10,EXCLUDED SINCE 2021,EXCLUDED SINCE 2021,NaN,0,0,218,2012,NOT SET,OPEN


In [36]:
df_compliance.nunique()

REGISTRY_CODE                       31
INSTALLATION_NAME                14324
INSTALLATION_IDENTIFIER           9290
PERMIT_IDENTIFIER                14450
MAIN_ACTIVITY_TYPE_CODE             36
COMPLIANCE_CODE                      6
CH_COMPLIANCE_CODE                   6
COMPLIANCE_STATUS_LATEST_YEAR        4
TOTAL_VERIFIED_EMISSIONS         11063
CH_TOTAL_VERIFIED_EMISSIONS        208
TOTAL_SURRENDERED_ALLOWANCES     10657
YEAR_OF_FIRST_EMISSIONS             20
YEAR_OF_LAST_EMISSIONS               7
ACCOUNT_CLOSURE                    255
dtype: int64

In [26]:
pd.read_excel(r"../data/raw/compliance_2020_code_en.xlsx" ,nrows=0, header=0).columns.to_list()

['REGISTRY_CODE',
 'INSTALLATION_NAME',
 'INSTALLATION_IDENTIFIER',
 'PERMIT_IDENTIFIER',
 'MAIN_ACTIVITY_TYPE_CODE',
 'COMPLIANCE_CODE',
 'CH_COMPLIANCE_CODE',
 'COMPLIANCE_STATUS_LATEST_YEAR',
 'TOTAL_VERIFIED_EMISSIONS',
 'CH_TOTAL_VERIFIED_EMISSIONS',
 'TOTAL_SURRENDERED_ALLOWANCES',
 'YEAR_OF_FIRST_EMISSIONS',
 'YEAR_OF_LAST_EMISSIONS',
 'ACCOUNT_CLOSURE']

## Verified enmission
- Unique id = REGISTRY_CODE + INSTALLATION_IDENTIFIER

In [ ]:
pd.read_excel(r"../data/raw/verified_emissions_2024_en.xlsx" , nrows=0, header=20)

In [ ]:
df_map_acts = pd.read_excel(r"../data/raw/verified_emissions_2024_en.xlsx" , sheet_name=1)

In [25]:
df_map_acts

,value,code
0,Combustion installations with a rated thermal ...,1
1,Mineral oil refineries,2
2,Coke ovens,3
3,Metal ore (including sulphide ore) roasting or...,4
4,Installations for the production of pig iron o...,5
5,Installations for the production of cement cli...,6
6,Installations for the manufacture of glass inc...,7
7,Installations for the manufacture of ceramic p...,8
8,Industrial plants for the production of (a) pu...,9
9,Aircraft operator activities,10


In [19]:
df_vemiss = pd.read_excel(r"../data/raw/verified_emissions_2024_en.xlsx", header=20)

In [ ]:
df_vemiss

In [46]:
df_ids = pd.read_excel(r"../data/raw/verified_emissions_2024_en.xlsx" , usecols=['REGISTRY_CODE',
 'IDENTIFIER_IN_REG',
 'INSTALLATION_NAME',
 'INSTALLATION_IDENTIFIER',
 'PERMIT_IDENTIFIER'], header=20)

In [49]:
df_ids[df_ids[['INSTALLATION_IDENTIFIER', 'REGISTRY_CODE']].duplicated(keep=False)].sort_values('INSTALLATION_IDENTIFIER')

,REGISTRY_CODE,IDENTIFIER_IN_REG,INSTALLATION_NAME,INSTALLATION_IDENTIFIER,PERMIT_IDENTIFIER


In [3]:
pd.read_excel(r"../data/raw/verified_emissions_2024_en.xlsx" , usecols=['REGISTRY_CODE',
 'IDENTIFIER_IN_REG',
 'INSTALLATION_NAME',
 'INSTALLATION_IDENTIFIER',
 'PERMIT_IDENTIFIER'], header=20).nunique()

REGISTRY_CODE                 32
IDENTIFIER_IN_REG          16246
INSTALLATION_NAME          17075
INSTALLATION_IDENTIFIER     9584
PERMIT_IDENTIFIER          14692
dtype: int64

### Emissions - Free EUAs

In [11]:
df_vemiss[['REGISTRY_CODE',
 'IDENTIFIER_IN_REG',
 'INSTALLATION_NAME',
 'INSTALLATION_IDENTIFIER',
'PERMIT_IDENTIFIER']][df_vemiss.duplicated(subset = ['INSTALLATION_IDENTIFIER', 'IDENTIFIER_IN_REG'], keep=False)].sort_values(by ='INSTALLATION_IDENTIFIER')

,REGISTRY_CODE,IDENTIFIER_IN_REG,INSTALLATION_NAME,INSTALLATION_IDENTIFIER,PERMIT_IDENTIFIER
10078,GB,Operator Account,Rough 47/3B,2,GB-DTI0200
12193,IT,Operator Account,Centrale a Biomasse C&T - Airasca,2,IT-A-2
11721,IE,Operator Account,Kingscourt Works,2,IE-GHG002
12734,IT,Operator Account,IMPIANTO DI COG. E TELERIS. DI BARDONECCHIA,3,IT-A-3
11618,IE,Operator Account,Arrabawn Cooperative Society Limited,3,IE-GHG003
...,...,...,...,...,...
13090,IT,Operator Account,S.p.A. BIRRA PERONI BARI,675,IT-A-1010
9594,GB,Operator Account,George Square,682,UK-S-IN-12354
12717,IT,Operator Account,IDEAL CART SPA,682,IT-A-1017
13102,IT,Operator Account,Sampol Italia srl,704,IT-A-1043


In [66]:
import re
import numpy as np
import pandas as pd

# Detect years from the emissions columns
years = sorted({
    re.search(r"(\d{4})$", c).group(1)
    for c in df_vemiss.columns
    if c.startswith("VERIFIED_EMISSIONS_") and re.search(r"\d{4}$", c)
})

for y in years:
    a = f"VERIFIED_EMISSIONS_{y}"
    e = f"ALLOCATION_{y}"
    net = f"NET_POSITION_{y}"

    # Coerce to numeric (in case there are strings)
    df_vemiss[e] = pd.to_numeric(df_vemiss[e], errors="coerce")
    df_vemiss[a] = pd.to_numeric(df_vemiss[a], errors="coerce")

    # Treat -1 as missing (blank / not reported)
    df_vemiss.loc[df_vemiss[e] == -1, e] = np.nan
    df_vemiss.loc[df_vemiss[a] == -1, a] = np.nan

    # If you have 'Excluded' values (usually in separate columns, but just in case)
    # convert them to NaN when present as strings
    # (this line is safe even if there are no strings)
    df_vemiss[e] = df_vemiss[e].replace("Excluded", np.nan)
    df_vemiss[a] = df_vemiss[a].replace("Excluded", np.nan)

    # Compute only when both are present; otherwise keep NaN
    df_vemiss[net] = df_vemiss[e] - df_vemiss[a]

In [67]:
# Get all net position columns automatically
net_cols = [col for col in df_vemiss.columns if col.startswith("NET_POSITION_")]

# Sum across years ignoring NaN
df_vemiss["TOTAL_NET_POSITION"] = df_vemiss[net_cols].sum(axis=1, skipna=True)


In [68]:
df_vemiss.groupby('MAIN_ACTIVITY_TYPE_CODE').agg(net_by_activity = ('TOTAL_NET_POSITION', 'sum'))

,net_by_activity
MAIN_ACTIVITY_TYPE_CODE,
1,-2.328770e+08
2,-2.429090e+07
4,1.565660e+05
5,1.301959e+06
6,2.467718e+07
7,-7.482250e+05
8,1.857608e+06
9,1.082914e+07
10,-1.858690e+08


### by activity

In [69]:
import pandas as pd

# 1️⃣ Detect year columns
net_year_cols = [
    c for c in df_vemiss.columns
    if c.startswith("NET_POSITION_")
]

# 2️⃣ Groupby sum for TOTAL + each year
agg_dict = {col: "sum" for col in ["TOTAL_NET_POSITION"] + net_year_cols}

df_net_by_activity = (
    df_vemiss
    .groupby("MAIN_ACTIVITY_TYPE_CODE", dropna=False)
    .agg(agg_dict)
    .reset_index()
)

# 3️⃣ Rename year columns to just the year (cleaner)
rename_dict = {}

for col in net_year_cols:
    year = col.split("_")[-1]   # safer than regex
    rename_dict[col] = year

df_net_by_activity = df_net_by_activity.rename(columns=rename_dict)

# 4️⃣ Merge activity names
df_net_by_activity["MAIN_ACTIVITY_TYPE_CODE"] = pd.to_numeric(
    df_net_by_activity["MAIN_ACTIVITY_TYPE_CODE"], errors="coerce"
)

df_map_acts["code"] = pd.to_numeric(df_map_acts["code"], errors="coerce")

df_net_by_activity = df_net_by_activity.merge(
    df_map_acts[["code", "value"]],
    left_on="MAIN_ACTIVITY_TYPE_CODE",
    right_on="code",
    how="left"
)

df_net_by_activity = df_net_by_activity.rename(columns={"value": "ACTIVITY_NAME"})
df_net_by_activity = df_net_by_activity.drop(columns=["code"])


In [70]:
df_net_by_activity.sort_values(by="TOTAL_NET_POSITION", ascending=False)


,MAIN_ACTIVITY_TYPE_CODE,TOTAL_NET_POSITION,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,ACTIVITY_NAME
13,24,7.033248e+08,50014810.0,86732275.0,66811331.0,67766370.0,70099677.0,39358209.0,33943769.0,32950039.0,35111775.0,27859677.0,24451638.0,25489114.0,35199601.0,20694308.0,27554300.0,36532263.0,22755642.0,Production of pig iron or steel
18,29,2.438940e+08,14105998.0,44021330.0,46565587.0,47479430.0,55910644.0,26549620.0,9897881.0,9693743.0,8990597.0,2389968.0,-2714194.0,-3128806.0,920858.0,-14524688.0,-7394393.0,2480111.0,2650339.0,Production of cement clinker
31,42,6.714697e+07,3187486.0,5474965.0,4504565.0,5458344.0,7040984.0,7873957.0,7447728.0,4430209.0,2994395.0,2246397.0,1474919.0,2988142.0,2099427.0,-1580734.0,3907127.0,6248661.0,1350394.0,Production of bulk chemicals
25,36,4.190696e+07,3077383.0,5925100.0,5645804.0,6515305.0,8187974.0,3470896.0,3467448.0,2384368.0,1681413.0,168917.0,-307214.0,-782442.0,-252481.0,-1598899.0,-70204.0,2731235.0,1662358.0,Production of paper or cardboard
14,25,3.088939e+07,2815784.0,5610807.0,8944572.0,10245448.0,10387797.0,2350363.0,2421416.0,130649.0,-1178560.0,-2019281.0,-2007392.0,-1486028.0,297135.0,-2184783.0,-1516369.0,-790512.0,-1131659.0,Production or processing of ferrous metals
24,35,2.687571e+07,2290379.0,2800977.0,2595529.0,2931117.0,2962059.0,1675525.0,1555496.0,1205949.0,934253.0,816288.0,643016.0,753493.0,718517.0,854174.0,1257634.0,1572184.0,1309116.0,Production of pulp
4,6,2.467718e+07,1264810.0,4583902.0,5087841.0,6065732.0,7278058.0,1338118.0,-57790.0,101462.0,-30226.0,-270447.0,-209000.0,-203823.0,-189692.0,-308980.0,-127306.0,55127.0,299391.0,Installations for the production of cement cli...
21,32,1.461380e+07,3094711.0,6282288.0,6863610.0,6383113.0,7424182.0,1502089.0,488467.0,-168438.0,-911332.0,-1837250.0,-2391945.0,-2592429.0,-2031617.0,-3877835.0,-2723334.0,-171463.0,-719021.0,Manufacture of ceramics
33,44,1.219242e+07,-65844.0,170395.0,58818.0,4729.0,-24511.0,2491007.0,2214479.0,1572292.0,1452245.0,1306256.0,1269540.0,-75747.0,531530.0,50023.0,45561.0,795021.0,396627.0,Production of soda ash and sodium bicarbonate
7,9,1.082914e+07,521739.0,856448.0,644834.0,725721.0,1017128.0,829569.0,742759.0,716080.0,580000.0,435605.0,453435.0,431515.0,401782.0,457536.0,712206.0,727364.0,575422.0,Industrial plants for the production of (a) pu...


In [71]:
df_net_by_activity.sort_values(by="TOTAL_NET_POSITION", ascending=False).to_excel("../data/processed/net_positions_by_activity.xlsx", index=False)


## Account holders

In [36]:
pd.read_excel(r"../data/raw/policy_ets_registry_operators_ets_en.xlsx", nrows=0, header=0).columns.to_list()

['Account Holder Name',
 'Company Registration Nr of Account Holder',
 'LEI',
 'MS Registry',
 'Installation ID',
 'Installation Name',
 'Activity Type',
 'Permit ID',
 'PERMIT_REVOCATION_DATE',
 'Permit Expiry/Revocation Date ',
 'Contact Country',
 'Contact City',
 'Contact PCode',
 'Contact Address L1',
 'Contact Address L2']

In [37]:
pd.read_excel(r"../data/raw/policy_ets_registry_operators_ets_en.xlsx",
              usecols=['Account Holder Name', 'Company Registration Nr of Account Holder', 'LEI']).nunique()

Account Holder Name                          10170
Company Registration Nr of Account Holder     9651
LEI                                           2033
dtype: int64

In [57]:
df_ah = pd.read_excel(r"../data/raw/policy_ets_registry_operators_ets_en.xlsx", header=0)

In [64]:
m = df_ah[df_ah.duplicated(subset = ['Company Registration Nr of Account Holder'], keep=False)].sort_values('Company Registration Nr of Account Holder')

In [ ]:
m